# Реализация системы распознавания лиц
## Дипломный проект Гринева Дмитрия
![](https://3dnews.ru/assets/external/illustrations/2018/02/08/965342/sm.thumbnail_1518065961.750.jpg)

Трудно себе представить современный мир без искуственного интеллекта. Глубокие нейросети проникли во все сферы бизнеса и получили широкое применение в задачах компьютерного зрения (__Computer Vision__) к которым относятся системы идентификации по биометрическим признакам (лицу) или попросту - __системы распознавания лиц__ (FRS).

Системы распознавания лиц решают две задачи:
- __Верификация__ - 1 к 1 сравнение двух лиц между собой
- __Идентификация__ - 1 к n поиск в базе данных известных лиц

### Области применения
1. __Контроль доступа__ - Применяется для управления конечными устройствами системы (турникеты, шлагбаумы, калитки и т.п.) как самостоятельно так и в сочетании с двухфакторной аутентификацией пользователя (по пропуску, отпечатку пальца и т.д.)
2. __Онлайн шоппинг__ - Компания Alibaba планирует использовать свою платформу Alipay для совершения покупок через интернет. Компания уже запустила свою систему распознавания лиц __Smile to Pay__ в KFC города Ханчжоу. Система распознает лицо в течении пары секунд и затем верефицирует лицо, присылая соответствующее сообщение на мобильный телефон.
3. __Поиск преступников__ - Одна из самых популярных областей применения систем правоохранительными органами по всему миру. Сигналы с камер городского наблюдения, транспорта стекаются в ситуационные центры, где проводится поиск лиц по существующей базе преступников или нежелательных элементов и операторам выводятся соответствующие сработки для реагирования.
4. __Черный список__ - Может быть использована для отказа доступа на объект или общественное мероприятие лицам, ранее привлекаемых за хулиганство, неадекватное поведение и т.п.
5. __Организация фотографий__ - Можно группировать семейные фотографии, находить на них друзей, родственников и т.п.
6. __Посещаемость занятий__ - В Англии широкое применение системы распознавания лиц нашли в области отслеживания посещения студентами школ, вузов. Достаточно в конце учебного дня провести поиск по архиву распознанных за день лиц студентов и вывести тех, кого не оказалось в архиве.

## Пайплайн FRS
![](imgs/pipeline.jpg)

Пайплайн распознавания лиц может быть разделен на 3 основные части:
- **Детекция лица на изображении с ключевыми точками**
- **Выравнивание лица по ключевым точкам и обрезание до входного размера сети извлечения вектора лица**
- **Извлечение вектора лица с кропнутого изображения**
- Идентификация или верификация лица

### Методы детекции лица (Face Detection)

Метод детекции лиц используется для поиска лиц, присутствующих на данном изображении, извлечения лиц, если они есть, и обрезки лица для дальнейшего извлечения признаков. Существуют различные варианты алгоритмов для выполнения этой задачи в системе обнаружения / распознавания лиц.

### Методы, используемые при детекции лиц
**Каскады Хаара**: Детектор лиц на основе каскада Хаара был самым современным в обнаружении лиц в течение многих лет с 2001 года, когда он был представлен Виолой и Джонсом в их статье «Rapid Object Detection using a Boosted Cascade of Simple Features». За последние годы было сделано много улучшений.
Этот метод имеет простую архитектуру, которая работает на ЦП почти в реальном времени. Кроме того, он может обнаруживать изображения в разных масштабах. Но главный недостаток в том, что он дает ложные результаты, а также не работает с нефронтовыми изображениями.

**Dlib (HOG)**: это широко используемая модель распознавания лиц,
основан на функциях HoG и SVM, опубликованных в 2005 году в статье «Histograms of oriented gradients for human detection». HOG или гистограмма направленных градиентов - это дескриптор объекта, который часто используется для извлечения признаков из изображения. Это самый быстрый метод на ЦП, который может работать как с фронтальными, так и с немного не фронтальными изображениями.
Но он не способен обнаруживать маленькие изображения и обрабатывать окклюзии. Кроме того, при обнаружении часто исключаются некоторые части подбородка и лба.

**Dlib (CNN)**: этот метод, впервые представленный в статье 2016 г. «CNN based efficient face recognition technique using Dlib», использует Maximum-Margin Object Detector (MMOD) с признаками, полученными при помощи сверточных нейронных сетей. Процесс обучения этого метода очень прост, и вам не потребуется большой объем данных для обучения пользовательского детектора объектов. Он очень быстро работает на GPU и способен работать с различными положениями лиц в кадре. Но основным недостатком является то, что он обучен минимальному размеру лица 80 * 80, поэтому он не может обнаруживать маленькие лица на изображениях. Также у данного метода медленный инференс на CPU.

**MTCNN**: Multi-task Cascaded Convolutional Networks (MTCNN) - это фреймворк, разработанный для решения как задачии обнаружения лиц в кадре, так и их выравнивания.
Этот метод был впервые представлен в статье под названием «Joint Face Detection and Alignment using Multi-task Cascaded Convolutional Networks» в 2016 году. Этот метод дает наиболее точные результаты из всех четырех методов. Он работает с различными положениями лиц на изображениях, и может обнаруживать их в различных масштабах. Он не имеет серьезных недостатков как таковых, но работает сравнительно медленнее, чем HOG и каскады Хаара. Долгое время MTCNN был **sota** решением в детекции лиц, пока ему на смену в 2019 году не пришла **RetinaFace**, которая по сей день является для многих является **baseline** в детекции лиц.

**RetinaFace**: RetinaFace: Single-shot Multi-level Face Localisation in the Wild - как видно из названия, за основу был взял детектор объектов **RetinaNet**, который был адаптирован под задачу детекции лиц и предсказания 5 ключевых точек лица, а также 3D реконструкции маски лица. Данный детектор показывает точность на Wider Face (HARD) **0.914**, что является sota и по сей день.

### Выравнивание лица (Face Aligment)

Выравнивание лица является неотъемлимой частью пайплайна и выполняется перед передачей полученного из кадра лица в сеть извлечения признаков - **эмбеддинга** лица, т.е. вектора заданной размерности. В своих исследованиях Google обнаружили, что выравнивание лиц повышает точность модели распознавания FaceNet с 98,87% до 99,63%. Это увеличение точности почти на 1 процент.
Для выравнивания лица достаточно получить координаты глаз и использовать правило косинуса по формуле:

![](https://i2.wp.com/sefiks.com/wp-content/uploads/2020/02/angelina-cosine-rule.jpg?resize=560%2C325&ssl=1)

**cos(A) = (b2 + c2 – a2 ) / (2bc)**

После этого спокойно получаем угол, на который нужно повернуть изображение, используя простой код:
`
angle = np.arccos(cos_a)
angle = (angle * 180) / math.pi # переводим из радиан в градусы

`
    
Кроме того, выравнивание лица может быть выполнено аффинным преобразованием, например, как сделали в библиотеке **insightface**:

    src = np.array([ 
        [30.2946, 51.6963], 
        [65.5318, 51.5014], 
        [48.0252, 71.7366], 
        [33.5493, 92.3655], 
        [62.7299, 92.2041] ], dtype=np.float32 ) 
    if image_size[1]==112: 
        src[:,0] += 8.0 
    dst = landmark.astype(np.float32) 
    
    tform = trans.SimilarityTransform() 
    tform.estimate(dst, src) 

### Извлечение признаков лица (Feature Extraction)

Извлечение признаков является основным и наиболее важным этапом в пайплайне распознавания лиц. Он извлекает биометрические признаки вашего лица. Эти признаки - особенности вашего лица, которые различаются от человека к человеку. Существуют различные методы извлечения признаков (эмбеддингов). Обычно используются размеры эмбеддинга кратные 2 - 128, 256, 512 и т.п. 
Задача извлечения признаков лица сводится к тренировке такого бэкбона, который обеспечит нас удобными векторами лиц, которые можно сравнивать. Для тренировки используется функция потерь — математическая функция, которая говорит, насколько хорошо наша текущая сеть справляется с задачей. Значение функции — штраф (чем он меньше — тем лучше), а переменные — параметры сети.
В DNN сетях существует два основных подхода к задаче получение векторов лиц: обучение метрики (metric learning) и решение задачи классификации.

**Обучение метрики**

В случае обучения метрики, мы заставляем сеть увеличивать расстояние между фотографиями разных людей и уменьшать его между фотографиями одного и того же человека. Самый известный пример — **Triplet Loss**, которую предложили в 2015 году представители Google в своей работе **FaceNet**:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/933c19129ec9060b0e7ea6f54f715c4c92010399)

A - якорное изображение лица, P - позитивный пример того же человека, N - пример другого человека, alpha - некая константа, которая отвечает за минимальное расстояние между позитивной и негативной парами.

**Классификация**

Для решения задачи классификации необходимо добавить к нашей backbone сети полносвязный слой для классификации, затем обучить сеть классифицировать людей (каждый класс — отдельный человек), затем необходимо удалить этот слой, и окажется, что последний полносвязный слой, который был перед нашим классификационным слоем будет выдавать верные эмбеддинги лица, размер которых будет равен размеру полносвязного слоя.
На практике оказалось, что данный подход показывает лучше результаты чем triplet loss и все современные sota решения обучены именно с применением этого подхода.

Классический подход к обучению сети в задаче классификации является функция потерь Softmax Loss, она же CrossEntropy + Softmax activation:

![](https://habrastorage.org/getpro/habr/formulas/ee1/f47/6d6/ee1f476d6798519f6dd4ab116d9fa32f.svg)

Обучение с обычным softmax loss имеет одну очень важную проблему - сеть ровно разделит классы между собой и у каждого человека будет свое пространство, в которое будут попадать векторы его лица (в идеале). Границы этих пространств могут быть близки друг к другу. Расстояние (угол) между изображениями на границах близких классов может быть меньше, чем расстояние между некоторыми изображениями одного класса, а это серьезная проблема.

Первым шагом к решению данной проблемы стала функция потерь Normalized Softmax:

![](https://habrastorage.org/getpro/habr/formulas/d52/5ab/262/d525ab262a306fc24a7a3cb5ba41809d.svg)

Здесь вводится понятие нормы лица **scale** ![](https://habrastorage.org/getpro/habr/formulas/ff5/d1a/01d/ff5d1a01d9102761a0a84024b068caaa.svg) - первый гиперпараметр функции потерь.

Далее появилась идея добавить между пространствами классов некоторую пустую область (decision margin). Размер этой области — margin — второй главный гиперпараметр функции потерь. Графически на нашей 2D гиперсфере это выглядит так:

![](https://habrastorage.org/r/w1560/webt/kp/ic/vd/kpicvd1vebzshxykvcwuapx67q4.jpeg)
<br>
Рисунок из статьи [https://arxiv.org/abs/1801.07698](ArcFace) Слева - без margin, справа - с margin.
</br>

Теперь между классами есть пустая область, и граничные изображения одного класса далеко до изображений других классов. В данном случае речь идет об углах между векторами, поэтому margin (обозначим $m$) можно добавить в три места в функцию потерь:

1. Домножить на угол. Данный вариант использовался в одной из первых работ Margin-based loss **SphereFace**. Функция потерь выглядит так:

![](https://habrastorage.org/getpro/habr/formulas/e46/aa1/066/e46aa106627de0e9e1a4b4ea5c3f9253.svg)

2. Отнять margin от косинуса угла. Данный подход был применен в двух работах: **CosFace** и **AM-SoftMax**:

![](https://habrastorage.org/getpro/habr/formulas/ffe/baa/e81/ffebaae81d7ddfa001d7b36291f95721.svg)

3. Прибавить margin к углу. Данная идея реализована в текущем sota подходе **ArcFace**:

![](imgs/arcface.jpg)

В обзоре https://arxiv.org/pdf/1901.05903 тесты на **LFW** показывают следующие результаты:
<table>
<thead>
<tr>
<th>Loss</th>
<th>Resnet50-MSC</th>
<th>MobileNet-MSC</th>
<th>Resnet50-Casia</th>
<th>MobileNet-Casia</th>
</tr>
</thead>
<tbody>
<tr>
<td>AM-Softmax/CosFace</td>
<td><strong>99.3</strong></td>
<td>97.65</td>
<td>99.34</td>
<td>98.46</td>
</tr>
<tr>
<td>ArcFace</td>
<td>99.15</td>
<td><strong>98.43</strong></td>
<td><strong>99.35</strong></td>
<td><strong>99.01</strong></td>
</tr>
<tr>
<td>SphereFace</td>
<td>99.02</td>
<td>96.86</td>
<td>99.1</td>
<td>97.83</td>
</tr>
</tbody>
</table>

В статье https://arxiv.org/pdf/1804.06655.pdf приводится история развития функций потерь:

![](imgs/losses_history.png)

Использование функции потерь ArcFace и по сей день является одним из sota подходов в т.н. **represenation stage** пайплайна распознавания лиц.

### Нейросетевые методы извлечения признаков лица (face embeddings)

В 2014 и 2015 годах было множество исследований и публикаций по методам глубокого обучения для распознавания лиц. Сверточные сети быстро достигли точности, близкой к человеческому уровню, а затем превосходили его в течении трех лет на стандартном датасете для распознавания лиц - **Labeled Faces In The Wild**.
Ключевыми игроками в вопросе распознавания лиц были: DeepFace, DeepID, VGGFace и FaceNet.

**VGGFace**: дескрипторы лиц в VGG-Face вычисляются с использованием реализации CNN на основе VGG-Very-Deep-
16 CNN и валидируются на наборах данных **LFW** и **YouTube Faces**. VGG использует различные архитектуры, такие как VGGFace1, VGGFace2 от Keras. Основное различие между этими моделями заключается в количестве слоев, включенных в их архитектуру, которое варьируется от модели к модели.

**FaceNet**: FaceNet - модель, разработанная в 2015 году исследователями Google в их статье 2015 года под названием «FaceNet: A Unified Embedding for Face Recognition and Clustering».
В данной работе исследователи Google предложили новую функцию потерь - **Triplet loss**, которая позволила обучать непосредственно embedding лица. В момент появления данная модель показала sota результаты с 128-байтовым вектором лица. На наборах данных **LFW** модель достигла точности 99.63%, а на **YouTube Faces DB** - 95.12%.

**DeepFace**: DeepFace - была описана в документе 2014 года «DeepFace: Closing the Gap to Human-Level Performance in Face Verification». В своей работе авторы уделили особое внимание шагу выравнивания лица и использовали 3D-моделирование лица для последующего аффинного преобразования. На датасете **LFW** они получили точность 97,35%.

**DeepID**: DeepID - это серия систем (например, DeepID, DeepID2 и т.д.). Описана в статье 2014 года «Deep Learning Face Representation from Predicting 10,000 Classes». Данная сеть предсказывает 5 точек лица, по которым идет выравнивание лица аффинным преобразованием (используются центры глаз и точка между уголков рта). Эмбеддинги извлекаются из 60 патчей лиц с 10 областями, тремя масштабами, в RGB или GRAYSCALE. Авторы обучали 60 ConvNets, каждая из которых извлекала два 160-мерных вектора DeepID из определенного патча и его горизонтально развернутый аналог. Общая длина вектора DeepID - 19,200 (160х2х60), который далее подается на верификацию с использованием подхода, описанного в статье 2012 года «Bayesian face revisited: A joint formulation». Достигли точности 97,45% на **LFW**.

**OpenFace**: OpenFace - фрэймворк, описанный в 2016 года в статье «OpenFace: An open source facial behavior analysis toolkit». Пайплайн данного фрэймворка включает в себя детектор лица, предсказание 68 ключевых точек лица, оценку направления взгляда, оценку положения головы, выравнивание лица, оценку выражения лица (action units). Из выравненного лица размера 112х112 извлекаются дескрипторы HOG и получается 4464-D вектор, который потом сжимается при помощи PCA до размерности 1391. Далее можно использовать SVM для классификации.

**InsightFace**: InsightFace - open-source решение для аналитики в области распознавания лиц основанный на Pytorch и MXNet. Авторы детектора **RetinaFace** и функции потерь **ArcFace**. На текущий момент это лучший **baseline** для создания системы распознавания лиц. У авторов есть хороший зоопарк обученных моделей, который можно взять под свои нужды, либо взять имплементации на pytorch, tf2 и обучить на своем датасете.

### Идентификация лиц

Список используемой литературы:

https://medium.com/backprop-labs/face-recognition-pipeline-clearly-explained-f57fc0082750

https://sefiks.com/2020/02/23/face-alignment-for-face-recognition-in-python-within-opencv/

https://habr.com/ru/company/ntechlab/blog/531842/